# Case-Control Study (Multiple Exposures): Sleeping Position and Sudden Infant Death Syndrome

## Summary

## Introduction

## Method

## Analysis

In [1]:
from opyn.generic.pandasloader import PandasLoader
from opyn.stats import epidemeology as epyn
from pandas.api.types import CategoricalDtype

### Prepare the data

In [2]:
pdloader = PandasLoader()
f = "sids"
pdloader.get_description(f)

title: Cot deaths and sleeping position

description
    Results of a case-control study looking at the association between
    the sleeping position of an infant and Sudden Infant Death
    Syndrome (SIDS).

reference: https://doi.org/10.1136/bmj.301.6743.85

fields:
    count:
        type: int
        desc: Number of observations
    exposure:
        type: str
        desc: Sleeping position of infant.
        values:
            - slept on front
            - slept on side
            - slept on back
    outcome
        type: str
        desc: Case or control group
        values:
            - case
            - control  



In [3]:
dat = pdloader.get(f)

Copy the dataframe to preserve the immutable nature of the data, and then recode the `exposure` and `outcome` columns as ordered categorical data.

In [4]:
copieddat = dat.copy(deep=True)
copieddat["exposure"] = copieddat["exposure"].astype(
    CategoricalDtype(["slept on side", "slept on front", "slept on back"], True)
)
copieddat["outcome"] = copieddat["outcome"].astype(
    CategoricalDtype(["control", "case"], True)
)

Sort the dataframe to ensure the data is as expected.

In [5]:
sorteddat = copieddat.sort_values(by=["exposure", "outcome"])

Extract the `count` column as a `2x2` `ndarray`.

In [6]:
resarr = sorteddat["count"].to_numpy().reshape((3, 2))
resarr

array([[241,  76],
       [ 24,  30],
       [509,  82]], dtype=int64)

It is this new reshaped `ndarray` that we will pass to the various functions for analysis.

### Measures of Association

In [7]:
epyn.oddsratio(resarr)

,oddsratio,stderr,lower,upper
Exposed1 (-),1.0,0.0,NA,NA
Exposed2 (+),3.963816,0.303821,2.185247,7.18996
Exposed3 (+),0.510857,0.177389,0.360832,0.723259


### Chi-squared test of no association

In [8]:
epyn.expectedfreq(resarr)

array([[255.04989605,  61.95010395],
       [ 43.44698545,  10.55301455],
       [475.5031185 , 115.4968815 ]])

In [9]:
epyn.chisqcontribs(resarr)

array([[ 0.77396455,  3.18642854],
       [ 8.70452205, 35.8367025 ],
       [ 2.35969235,  9.7149036 ]])

In [10]:
epyn.chisqtest(resarr)

,chisq,pval,df
result,60.576214,7.015237e-14,2.0


## Discussion